# Titanic

## Librerie

In [361]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sl

from tensorflow import keras
from sklearn.model_selection import train_test_split

print(tf.__version__)
print(pd.__version__)
print(np.__version__)
print(sl.__version__)

2.2.0
1.0.4
1.18.4
0.22.2.post1


## Download dei Dataset

Importiamo i dataset necessari per l'addestramento e per il test della rete neurale:

In [0]:
url_train = 'https://raw.githubusercontent.com/rirolli/Titanic/master/train.csv'
url_test = 'https://raw.githubusercontent.com/rirolli/Titanic/master/test.csv'
url_test_y = 'https://raw.githubusercontent.com/rirolli/Titanic/master/gender_submission.csv'

titanic_load_train = pd.read_csv(url_train)
titanic_load_test = pd.read_csv(url_test)
titanic_load_test_label = pd.read_csv(url_test_y)

In [363]:
# Stampa delle prima 5 righe del dataset di train
titanic_load_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Ottimizzazione del Dataset

Preparazione dei dati rendendoli tutti in forma numerica in quanto il modello della rete neurale di tensorflow esegue calcoli solo su numeri e non sulle stringhe. Inoltre eliminiamo la colonna 'Name' poiché non utile ai fini del problema.

In [0]:
# rimozione della colonna dei nomi delle persone in quanto non è necessaria
# ai fini dell'apprendimento della rete
titanic_load_train.pop('Name')
titanic_load_test.pop('Name')

# Codifica di tutti i dati dei dataset titanic_load_train e titanic_load_test
for elem_train in titanic_load_train:
  titanic_load_train[elem_train] = (pd.Categorical(titanic_load_train[elem_train])).codes

for elem_test in titanic_load_test:
  titanic_load_test[elem_test] = (pd.Categorical(titanic_load_test[elem_test])).codes


In [365]:
# Stampa delle prima 5 righe del dataset di train
titanic_load_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,0,2,1,28,1,0,523,18,-1,2
1,1,1,0,0,51,1,0,596,207,81,0
2,2,1,2,0,34,0,0,669,41,-1,2
3,3,1,0,0,47,1,0,49,189,55,2
4,4,0,2,1,47,0,0,472,43,-1,2


In [366]:
# Stampa delle prima 5 righe del dataset di test
titanic_load_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,2,1,44,0,0,152,24,-1,1
1,1,2,0,60,1,0,221,5,-1,2
2,2,1,1,74,0,0,73,41,-1,1
3,3,2,1,34,0,0,147,34,-1,2
4,4,2,0,27,1,1,138,46,-1,2


### Ripartizione del Dataset

Si esegue lo Split dei dataset per ricavare i dati di test e di valutazione: 

In [0]:
# Dividiamo il train set in due parti così da poter addestrare la rete:
# y che contiene solo la colonna 'Survived' che consiste nella soluzione al problema
# X che contiene tutte le altre colonne.
y = titanic_load_train.Survived
X = titanic_load_train.drop(labels=['Survived'], axis=1)

# I Dati vengono, a loro volta, divisi in altre due parti: una per
# l'addestramento e una per la valutazione della rete.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2)

# I dati vengono poi memorizzati dentro un Dataset di Tensorflow per sfruttarli
# per l'addestramento. Questa stessa operazione viene poi fatta anche per il
# test set e per l'evaluetion set.
train_dataset = (tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))).shuffle(len(X_train)).batch(1)
val_dataset = (tf.data.Dataset.from_tensor_slices((X_val.values, y_val.values))).shuffle(len(X_val)).batch(1)
test_dataset = (tf.data.Dataset.from_tensor_slices(titanic_load_test.values)).batch(1)

## La rete neurale

Il modello utilizzato è formato da 3 layer totalmente connessi e viene utilizzato come algoritmo di ottimizzazione l'adam:

In [0]:
num_inputs = X_train.shape[1]

In [369]:
model = keras.Sequential([
  tf.keras.layers.Dense(32, input_dim=num_inputs, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(1)
])

model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 32)                352       
_________________________________________________________________
dense_71 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 33        
Total params: 1,441
Trainable params: 1,441
Non-trainable params: 0
_________________________________________________________________


## Addestramento della rete

Ora è il momento di addestrare il modello tramite i dati di train:

In [370]:
# Addestramento della rete neurale
model.fit(train_dataset, epochs=300)

Epoch 1/300
712/712 [==============================] - 1s 1ms/step - loss: 3.4845 - accuracy: 0.5758
Epoch 2/300
712/712 [==============================] - 1s 1ms/step - loss: 2.0343 - accuracy: 0.6292
Epoch 3/300
712/712 [==============================] - 1s 1ms/step - loss: 1.6810 - accuracy: 0.6503
Epoch 4/300
712/712 [==============================] - 1s 1ms/step - loss: 1.4431 - accuracy: 0.6713
Epoch 5/300
712/712 [==============================] - 1s 1ms/step - loss: 1.2063 - accuracy: 0.6699
Epoch 6/300
712/712 [==============================] - 1s 1ms/step - loss: 1.3497 - accuracy: 0.6559
Epoch 7/300
712/712 [==============================] - 1s 1ms/step - loss: 1.0675 - accuracy: 0.6868
Epoch 8/300
712/712 [==============================] - 1s 1ms/step - loss: 0.9762 - accuracy: 0.6756
Epoch 9/300
712/712 [==============================] - 1s 1ms/step - loss: 1.1465 - accuracy: 0.6728
Epoch 10/300
712/712 [==============================] - 1s 1ms/step - loss: 0.8720 - accura

## Valutazione della rete

Eseguiamo la valutazione del modello:

In [371]:
# Valutazione del modello
val_loss, val_acc = model.evaluate(val_dataset)
print("\nTest accuracy: {:.2f} ({:.2%})".format(val_acc, val_acc))

179/179 [==============================] - 0s 2ms/step - loss: 1.1364 - accuracy: 0.7709

Test accuracy: 0.77 (77.09%)


## Previsione

Ora è il momento di predire i valori del dataset di test tramite l'addestramento appena effettuato:

In [0]:
# Predizione dei dati tramite il modello
predictions = model.predict(test_dataset)

Effettuiamo una breve stampa dei valori ottenuti tramite `predict` e degli effettivi valori riferimento. Qui vengono mostrato anche il valore di accuratezza della predizione:

In [373]:
r=0
f=0

# Eseguiamo la stampa dei valori 'predictions' ottenuti tramite
# la predizione con la rete neurale e delle etichette
# reali 'titanic_load_test_label['Survived']'.
print("passegerID \t Prediction \t\t\t Label \t Description\n")
for pid, prediction, label in zip(titanic_load_test['PassengerId'], tf.sigmoid(predictions).numpy(), titanic_load_test_label['Survived']):
  message = "{} \t\t Predicted survival: {:.2%} \t {}".format(pid, prediction[0], label)
  if tf.round(prediction)==label:
    r+=1
  else:
    message += " \t PREDIZIONE ERRATA"
    f+=1
  print(message)

print("\n - Totale predizioni corrette: {}\n - Totale predizioni errate: {}".format(r,f))

passegerID 	 Prediction 			 Label 	 Description

0 		 Predicted survival: 0.32% 	 0
1 		 Predicted survival: 2.92% 	 1 	 PREDIZIONE ERRATA
2 		 Predicted survival: 0.00% 	 0
3 		 Predicted survival: 0.02% 	 0
4 		 Predicted survival: 0.18% 	 1 	 PREDIZIONE ERRATA
5 		 Predicted survival: 0.00% 	 0
6 		 Predicted survival: 20.60% 	 1 	 PREDIZIONE ERRATA
7 		 Predicted survival: 0.00% 	 0
8 		 Predicted survival: 48.53% 	 1 	 PREDIZIONE ERRATA
9 		 Predicted survival: 0.00% 	 0
10 		 Predicted survival: 0.40% 	 0
11 		 Predicted survival: 0.00% 	 0
12 		 Predicted survival: 100.00% 	 1
13 		 Predicted survival: 0.00% 	 0
14 		 Predicted survival: 0.35% 	 1 	 PREDIZIONE ERRATA
15 		 Predicted survival: 0.01% 	 1 	 PREDIZIONE ERRATA
16 		 Predicted survival: 0.00% 	 0
17 		 Predicted survival: 0.03% 	 0
18 		 Predicted survival: 0.01% 	 1 	 PREDIZIONE ERRATA
19 		 Predicted survival: 0.00% 	 1 	 PREDIZIONE ERRATA
20 		 Predicted survival: 0.00% 	 0
21 		 Predicted survival: 0.00% 	 0
22 		

## Confusion Matrix

Tramite i valori ottenuti è possibile generare una confuion_matrix:



In [0]:
predictions_rounded = tf.round(tf.sigmoid(predictions)).numpy()

In [375]:
print(confusion_matrix(titanic_load_test_label['Survived'], predictions_rounded, normalize=None))

[[245  21]
 [ 63  89]]
